# Depth Estimation

In [ ]:
%load_ext tensorboard

import os
import time
import math
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import models as models
import ops.depth_estimation.datasets as datasets
import ops.depth_estimation.trains as trains
import ops.depth_estimation.tests as tests

In [ ]:
tf.random.set_seed(0)

plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['font.size'] = 15
plt.rcParams['figure.titlesize'] = 25
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 13
plt.rcParams['lines.linewidth'] = 2

In [ ]:
# Paths
cwd = os.getcwd()
model_path = 'models_checkpoints'
dat_path = 'leaderboard/depth-estimation'

# NYU Depth V2
dataset_name = 'nyu_depth_v2'
img_size = 480 // 2, 640 // 2
crop_size = 480 // 2 - 30, 640 // 2 - 60
# dataset_root = ...
seq_root = 'G:/dataset/nyudv2/nyu_depth_v2_raw/'


In [ ]:
dataset_train, dataset_val, _ = datasets.dataset(img_size, crop_size, flip=True)

In [ ]:
for xs, ys in dataset_train.skip(200).batch(3).take(1):
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    for i in range(3):
        _ = axes[i][0].imshow(xs[i])
        axes[i][0].axis('off')
        
        im = axes[i][1].imshow(tf.squeeze(ys, axis=-1)[i], cmap='gray', vmin=0)
        fig.colorbar(im, ax=axes[i][1], fraction=0.035, pad=0.04)
        axes[i][1].axis('off')

        y_min, y_max = 0.0, tf.reduce_max(ys, axis=[1, 2], keepdims=True)
        mask = tf.math.logical_and(ys > y_min, ys < y_max)
        axes[i][2].imshow(tf.squeeze(mask, axis=-1)[i], cmap='gray', vmin=0, vmax=1)
        axes[i][2].axis('off')


In [ ]:
offset=(5, 0)
dataset_seq = datasets.dataset_seq(seq_root, img_size, offset, i=4)
dataset_test = dataset_seq.map(lambda xs, y: (xs[offset[0]], y))

In [ ]:
for xs, ys in dataset_seq.batch(1).skip(5).take(1):
    fig, axes = plt.subplots(1, 6, figsize=(20, 4))
    for i, ax in enumerate(axes[:-1]):
        ax.imshow(xs[0][i])
        ax.axis('off')
    im = axes[-1].imshow(tf.squeeze(ys[0], axis=-1), cmap='gray', vmin=0)
    fig.colorbar(im, ax=axes[-1], fraction=0.035, pad=0.04)
    axes[-1].axis('off')


In [ ]:
# DNN
model = models.UNet(1, name='u-net-dnn')
# model = models.SegNet(1, name='segnet-dnn')

# BNN
# model = models.UNet(1, rate=0.5, name='u-net-bnn')
# model = models.SegNet(1, rate=0.5, name='segnet-bnn')

# model.load_weights('%s/%s_%s' % (model_path, dataset_name, model.name + '_2')) 


In [ ]:
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = 'logs/gradient_tape/%s_%s/%s' % (dataset_name, model.name, current_time)
train_log_dir = '%s/train' % log_dir
test_log_dir = '%s/test' % log_dir
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    
print('Create TensorBoard Log dir: ', log_dir)

## Train

In [ ]:
epochs = 6
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

epochtime_metric = tf.keras.metrics.Mean(name='epoch_time')
loss_metric = tf.keras.metrics.Mean(name='train_loss')
mse_metric = tf.keras.metrics.Mean(name='train_mse')
nll_metric = tf.keras.metrics.Mean(name='train_nll')

for epoch in range(epochs):
    batch_time = time.time()
    loss, mse = trains.train_epoch(optimizer, model, dataset_train.skip(30000).take(10000), batch_size=10)
    epochtime_metric(time.time() - batch_time)
    loss_metric(loss)
    mse_metric(mse)
    
    if (epoch + 1) % 1 == 0:
        template = '(%.2f sec) Epoch: %d, Loss: %.4f, MSE: %.4f'
        print(template % (epochtime_metric.result(),
                          epoch,
                          loss_metric.result(),
                          mse_metric.result()))
        
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', loss_metric.result(), step=epoch)
            tf.summary.scalar('mse', mse_metric.result(), step=epoch)
        
        epochtime_metric.reset_states()
        loss_metric.reset_states()
        mse_metric.reset_states()

    if (epoch + 1) % 1 == 0:
        metrics = tests.test_vanilla(model,
                                     dataset_val, batch_size=10, 
                                     cutoffs=(0.0, 0.9), verbose=False)
        
        with test_summary_writer.as_default():
            tf.summary.scalar('nll', metrics[0], step=epoch)
            tf.summary.scalar('mse', metrics[1], step=epoch)


In [ ]:
model.save_weights('%s/%s_%s' % (model_path, dataset_name, model.name + '_5'))

## Test

In [ ]:
_ = tests.test_vanilla(model,
                       dataset_test, batch_size=1, 
                       var=0.4, cutoffs=(0.0, 0.9), verbose=True)

In [ ]:
_ = tests.test_sampling(model, 30, 
                        dataset_test, batch_size=10, 
                        var=0.4, cutoffs=(0.0, 0.9), verbose=True)

In [ ]:
_ = tests.test_temporal_smoothing(model, 0.8, 
                                  dataset_seq, batch_size=10, 
                                  var=0.4, cutoffs=(0.0, 0.9), verbose=True)

Visualize results:

In [ ]:
for xs, ys in dataset_train.skip(10).batch(1).take(1):
    ys_pred = tests.predict_vanilla(model, xs, var=0.05)
    ys_pred_mean, ys_pred_var = tf.split(ys_pred, [1, 1], axis=-1)
    
    mse = tf.math.square(ys_pred_mean - ys)
    rmse = tf.math.sqrt(mse)
        
    fig, axes = plt.subplots(1, 5, figsize=(16, 5))
    axes[0].imshow(xs[0])
    im = axes[1].imshow(tf.squeeze(ys, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[1], fraction=0.035, pad=0.04)
    im = axes[2].imshow(tf.squeeze(ys_pred_mean, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[2], fraction=0.035, pad=0.04)
    im = axes[3].imshow(tf.squeeze(rmse, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[3], fraction=0.035, pad=0.04)
    im = axes[4].imshow(tf.squeeze(ys_pred_var, axis=-1)[0], vmin=0, cmap='gist_yarg')
    fig.colorbar(im, ax=axes[4], fraction=0.035, pad=0.04)
    
    for ax in axes:
        ax.axis('off')
        

In [ ]:
for xs, ys in dataset_val.skip(20).batch(1).take(1):
    ys_pred = tests.predict_sampling(model, xs, 10, var=0.05)
    ys_pred_mean, ys_pred_var = tf.split(ys_pred, [1, 1], axis=-1)
    
    mse = tf.math.square(ys_pred_mean - ys)
    rmse = tf.math.sqrt(mse)
    
    fig, axes = plt.subplots(1, 5, figsize=(16, 5))
    axes[0].imshow(xs[0])
    im = axes[1].imshow(tf.squeeze(ys, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[1], fraction=0.035, pad=0.04)
    im = axes[2].imshow(tf.squeeze(ys_pred_mean, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[2], fraction=0.035, pad=0.04)
    im = axes[3].imshow(tf.squeeze(rmse, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[3], fraction=0.035, pad=0.04)
    im = axes[4].imshow(tf.squeeze(ys_pred_var, axis=-1)[0], vmin=0, cmap='gist_yarg')
    fig.colorbar(im, ax=axes[4], fraction=0.035, pad=0.04)
    
    for ax in axes:
        ax.axis('off')
    

In [ ]:
for xs, ys in dataset_seq.skip(100).batch(2).take(1):
    ys_pred = predict_temporal_smoothing(model, xs, l=0.8, var=0.05)
    ys_pred_mean, ys_pred_var = tf.split(ys_pred, [1, 1], axis=-1)
    
    mse = tf.math.square(ys_pred_mean - ys)
    rmse = tf.math.sqrt(mse)
    
    fig, axes = plt.subplots(1, 5, figsize=(16, 5))
    axes[0].imshow(xs[0][-1])
    im = axes[1].imshow(tf.squeeze(ys, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[1], fraction=0.035, pad=0.04)
    im = axes[2].imshow(tf.squeeze(ys_pred_mean, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[2], fraction=0.035, pad=0.04)
    im = axes[3].imshow(tf.squeeze(rmse, axis=-1)[0], vmin=0, vmax=tf.reduce_max(ys[0]), cmap='gist_gray')
    fig.colorbar(im, ax=axes[3], fraction=0.035, pad=0.04)
    im = axes[4].imshow(tf.squeeze(ys_pred_var, axis=-1)[0], vmin=0, cmap='gist_yarg')
    fig.colorbar(im, ax=axes[4], fraction=0.035, pad=0.04)
            
    for ax in axes:
        ax.axis('off')